Welcome to a competition powered by AutoDSC for Data Science Challenges! By Prof. Manoel Gadi!

PLEASE DO NOT RENAME THIS FILE!


Simply run this code and start competing today in the competion: 6aQ6IxU7Va

6aQ6IxU7Va details:
 - Description / Descripción: FRAUD MODELLING CHALLENGE - Predict which Credit Card Application is legitimate and which belongs to a fraudster instead.
 - Maximum number of daily attempts / Número máximo de intentos diarios: 10000
 - Creation date / Fecha de creación: 2020-06-10 11:36:52
 - Starting date / Fecha de inicio: 2023-04-10 00:00:00
 - Ending date / Fecha de fin: 2023-05-29 23:59:00
 - Minimum time between prediction submissions / Tiempo mínimo entre envíos de predicciones: 30

Of couse, to win the competition you should improve the starting model! So let's get to work!


In [81]:
print ("IMPORTING LIBRARIES...")
import pandas as pd


print ("LOADING DATASETS...")
try: # reading train csv from local file
    df_train = pd.read_csv("mfalonso__6aQ6IxU7Va__train.csv")
    df_train.head()
except: # reading train csv from the internet if it is the first time
    import urllib
    csv_train = urllib.request.urlopen("http://manoelutad.pythonanywhere.com/static/uploads/mfalonso__6aQ6IxU7Va__train.csv")
    csv_train_content = csv_train.read()
    with open("mfalonso__6aQ6IxU7Va__train.csv", 'wb') as f:
            f.write(csv_train_content)
    df_train = pd.read_csv("mfalonso__6aQ6IxU7Va__train.csv")

    
try: # reading test csv from local file
    df_test = pd.read_csv("mfalonso__6aQ6IxU7Va__test.csv")
    df_test.head()
except: # reading test csv from the internet if it is the first time
    import urllib
    csv_test = urllib.request.urlopen("http://manoelutad.pythonanywhere.com/static/uploads/mfalonso__6aQ6IxU7Va__test.csv")
    csv_test_content = csv_test.read()
    with open("mfalonso__6aQ6IxU7Va__test.csv", 'wb') as f:
            f.write(csv_test_content)
    df_test = pd.read_csv("mfalonso__6aQ6IxU7Va__test.csv")


IMPORTING LIBRARIES...
LOADING DATASETS...


In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [82]:
print ("STEP 1: DOING MY TRANSFORMATIONS...")
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)


STEP 1: DOING MY TRANSFORMATIONS...


In [83]:
print ("STEP 2: SELECTING CHARACTERISTICS TO ENTER INTO THE MODEL...")
def get_specific_columns(df, data_types, to_ignore = list(), ignore_target = False):
    columns = df.select_dtypes(include=data_types).columns
    if ignore_target:
        columns = filter(lambda x: x not in to_ignore, list(columns))
    return list(columns)

output_var = df_train.columns[-1]
in_model = get_specific_columns(df_train, ["float64", "int64"], [output_var], ignore_target = True)


STEP 2: SELECTING CHARACTERISTICS TO ENTER INTO THE MODEL...


Developing the Model

In [84]:
print ("STEP 3: DEVELOPING THE MODEL...")
X_train = df_train[in_model]
y_train = df_train[output_var]
X_test = df_test[in_model]

STEP 3: DEVELOPING THE MODEL...


# Model 1: Grid Search for Hyperparameter Tunning (best model)

In [79]:
from sklearn.model_selection import GridSearchCV
        # Define the parameter grid for hyperparameter tuning
param_grid = {
   'n_estimators': [500, 1000, 1500],  # Number of trees in the forest
    'max_depth': [75, 80, 90],  # Maximum depth of each tree
    'min_samples_split': [2, 5], # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2]
     # Weighting strategy for the classes
}
        # Create the model Classifier to be used (either the tree or the random forest) 
clf = RandomForestClassifier(random_state=0, class_weight='balanced')
        # Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='roc_auc', cv=5)
grid_search.fit(X_train, y_train)
        
print("Best Hyperparameters:", grid_search.best_params_)
        # Use the best model for prediction
best_model = grid_search.best_estimator_
pred_test = best_model.predict_proba(X_test)[:, 1]
pred_train = best_model.predict_proba(X_train)[:, 1]



Best Hyperparameters: {'max_depth': 75, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 500}
STEP 3: DEVELOPING THE MODEL...


# Model 2: Testing SMOTE

In [103]:
!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
clf = RandomForestClassifier(random_state=80, class_weight='balanced')
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
fitted_model = clf.fit(X_train_resampled, y_train_resampled)
pred_train = fitted_model.predict_proba(X_train_resampled)[:,1]
pred_test = fitted_model.predict_proba(X_test)[:, 1]

print ("STEP 3: DEVELOPING THE MODEL...")

STEP 3: DEVELOPING THE MODEL...


# Model 3: Logistic Regression

In [95]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs')
fitted_model = clf.fit(X_train, y_train)
pred_train = fitted_model.predict_proba(X_train)[:,1]
pred_test = fitted_model.predict_proba(X_test)[:,1]

print ("STEP 3: DEVELOPING THE MODEL...")

STEP 3: DEVELOPING THE MODEL...


/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Model 4: Decision Tree Classifier 

In [101]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0, class_weight="balanced") 
fitted_model = clf.fit(X_train, y_train)
pred_train = fitted_model.predict_proba(X_train)[:,1]
pred_test = fitted_model.predict_proba(X_test)[:,1]
#gives error, cannot submit these values

print ("STEP 3: DEVELOPING THE MODEL...")

STEP 3: DEVELOPING THE MODEL...


# Model 5: Random Forest Classifier

In [ ]:

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0, class_weight='balanced')

fitted_model = clf.fit(X_train, y_train)
pred_train = fitted_model.predict_proba(X_train)[:,1]
pred_test  = fitted_model.predict_proba(X_test)[:,1]

params = clf.get_params()

for param, value in params.items():
    print(f'{param}: {value}')
    
print ("STEP 3: DEVELOPING THE MODEL...")

# Model 6: Naive Bayes (worst model)

In [113]:

from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
fitted_model = clf.fit(X_train, y_train)
pred_train = fitted_model.predict_proba(X_train)[:,1]
pred_test  = fitted_model.predict_proba(X_test)[:,1]

print ("STEP 3: DEVELOPING THE MODEL...")

STEP 3: DEVELOPING THE MODEL...


In [114]:
print ("STEP 4: ASSESSING THE MODEL...")
# CALCULATING GINI PERFORMANCE ON DEVELOPMENT SAMPLE
from sklearn.metrics import roc_auc_score
gini_score = 2*roc_auc_score(y_train, pred_train)-1
print ("GINI DEVELOPMENT=", gini_score)


STEP 4: ASSESSING THE MODEL...
GINI DEVELOPMENT= 0.44455834114339265


WHAT IS GINI?
* watch this video for reference: https://youtu.be/MiBUBVUC8kE


In [87]:
print ("STEP 5: SUBMITTING THE RESULTS... DO NOT CHANGE THIS PART!")
import requests
from requests.auth import HTTPBasicAuth
df_test['pred'] = pred_test
df_test['id'] = df_test.iloc[:,0]
df_test_tosend = df_test[['id','pred']]

filename = "df_test_tosend.csv"
df_test_tosend.to_csv(filename, sep=',')
url = 'http://manoelutad.pythonanywhere.com/uploadpredictions/6aQ6IxU7Va'
files = {'file': (filename, open(filename, 'rb')),
         'ipynbcode': ('6aQ6IxU7Va.ipynb', open('6aQ6IxU7Va.ipynb', 'rb'))}


#rsub = requests.post(url, files=files)
rsub = requests.post(url, files=files, auth=HTTPBasicAuth("joaoferris", "sha256$iu5U38YE$244190648eceab7fa04d46bb1e9cb62e199ef0ef218134abc0b8512cf2d0ffbf"))
resp_str = str(rsub.text)
print ("RESULT SUBMISSION: ", resp_str)

STEP 5: SUBMITTING THE RESULTS... DO NOT CHANGE THIS PART!
RESULT SUBMISSION:  ERROR: enviado despues de fecha de cierre


In [112]:
print ("STEP 5: SUBMITTING THE RESULTS... DO NOT CHANGE THIS PART!")
import requests
from requests.auth import HTTPBasicAuth
df_test['pred'] = pred_test
df_test['id'] = df_test.iloc[:,0]
df_test_tosend = df_test[['id','pred']]

filename = "df_test_tosend.csv"
df_test_tosend.to_csv(filename, sep=',')
url = 'http://manoelutad.pythonanywhere.com/uploadpredictions/6aQ6IxU7Va'
files = {'file': (filename, open(filename, 'rb')),
         'ipynbcode': ('6aQ6IxU7Va.ipynb', open('6aQ6IxU7Va.ipynb', 'rb'))}

df_test['pred'].mean()

STEP 5: SUBMITTING THE RESULTS... DO NOT CHANGE THIS PART!


0.19395685877991292